<a href="https://colab.research.google.com/github/franciscogarate/cdiae/blob/main/notebooks/8_PCA_California_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de componentes principales con la base de datos de casas de California

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

Cargamos el dataset de casas de California de ejercicios anteriores

In [4]:
!git clone https://github.com/franciscogarate/cdiae

fatal: destination path 'cdiae' already exists and is not an empty directory.


In [5]:
df = pd.read_feather('cdiae/data/03_model_input/california_housing_clean.ftr')
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
1,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
2,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
3,4.0368,52.0,4.761658,1.103627,413.0,2.139896,37.85,-122.25,2.697
4,3.6591,52.0,4.931907,0.951362,1094.0,2.128405,37.84,-122.25,2.992


Estadísticas descriptivas:

In [8]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
count,16393.000000,16393.000000,16393.000000,16393.000000,16393.000000,16393.000000,16393.000000,16393.000000,16393.000000
mean,3.664875,29.491795,5.157438,1.047261,1270.523699,2.862434,35.668856,-119.627363,1.942280
std,1.448800,12.204684,1.044754,0.066532,627.220656,0.625743,2.139559,1.995430,0.964867
min,0.536000,1.000000,2.032738,0.866013,5.000000,1.161290,32.540000,-124.350000,0.149990
25%,2.563900,19.000000,4.425963,1.002770,812.000000,2.434066,33.940000,-121.820000,1.188000
50%,3.500000,30.000000,5.144289,1.043807,1158.000000,2.811881,34.290000,-118.600000,1.781000
75%,4.593800,38.000000,5.852735,1.088685,1635.000000,3.243553,37.730000,-118.030000,2.509000
max,8.011300,52.000000,8.452915,1.239521,3132.000000,4.560748,41.950000,-114.570000,5.000000


Verificamos si hay valores nulos por columna:

In [9]:
print(df.isnull().sum())

MedInc        0
HouseAge      0
AveRooms      0
AveBedrms     0
Population    0
AveOccup      0
Latitude      0
Longitude     0
target        0
dtype: int64


- **y** será nuestra variable dependiente: la que queremos analizar, modelizar o predecir.
- **X** será nuestra variable o variables explicativas (independientes): la que usamos para explicar o predecir.

In [10]:
y = df['target']
X = df.drop(columns='target')

### 1. Estandarizamos los datos antes de aplicar PCA
Transforma cada variable para que tenga media 0 y desvest 1.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### 2. Aplicamos la fórmula de PCA

In [ ]:
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

### 3. Analizamos la varianza explicada por cada componente

In [ ]:
explained_variance_ratio = pca.explained_variance_ratio_
explained_variance_ratio

In [ ]:
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
cumulative_variance_ratio

### 4. Visualizamos la varianza explicada por cada componente:


In [ ]:
plt.figure(figsize=(10, 6))
bars = plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio)
plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, 'r-o', linewidth=2)
plt.xlabel('Componente Principal')
plt.ylabel('Proporción de Varianza Explicada')
plt.title('Varianza Explicada por Componente Principal')
plt.xticks(range(1, len(explained_variance_ratio) + 1))
plt.grid(True)
# Añadir etiquetas de porcentaje a las barras
for i, bar in enumerate(bars):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{explained_variance_ratio[i]:.2%}',
             ha='center', va='bottom', rotation=0)
plt.show()

### 5. Determinar el número óptimo de componentes
Podemos elegir el número de componentes que expliquen al menos el 95% de la varianza

In [ ]:
n_components = np.argmax(cumulative_variance_ratio >= 0.95) + 1
print(f"\nNúmero de componentes para explicar al menos el 95% de la varianza: {n_components}")
print(f"Varianza acumulada explicada con {n_components} componentes: {cumulative_variance_ratio[n_components-1]:.4f}")

Aplicamos PCA con el número óptimo de componentes

In [ ]:
pca_optimal = PCA(n_components=n_components)
X_pca_optimal = pca_optimal.fit_transform(X_scaled)

In [ ]:
print(f"\nDimensión original del dataset: {X.shape}")
print(f"Dimensión después de PCA: {X_pca_optimal.shape}")

### 6. Visualizamos los datos en las primeras 2 componentes principales

In [ ]:
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='coolwarm', alpha=0.5)
plt.colorbar(scatter, label='Precio de la casa (en $100,000)')
plt.xlabel('Primera Componente Principal')
plt.ylabel('Segunda Componente Principal')
plt.title('Proyección de los datos en las dos primeras componentes principales')
plt.grid(True)
plt.show()

### 7. Analizamos la contribución de cada característica original a las componentes principales

In [ ]:
loadings = pca.components_
feature_names = X.columns
loadings_df = pd.DataFrame(loadings.T, columns=[f'PC{i+1}' for i in range(loadings.shape[0])], index=feature_names)
loadings_df

Por último, visualizamos la contribución de cada característica a las dos primeras componentes principales

In [ ]:
for i, feature in enumerate(feature_names):
    plt.arrow(0, 0, loadings[0, i], loadings[1, i], head_width=0.05, head_length=0.05, fc='blue', ec='blue')
    plt.text(loadings[0, i], loadings[1, i], feature, fontsize=12)
    plt.xlabel('Primera Componente Principal')
    plt.ylabel('Segunda Componente Principal')
    plt.title('Contribución de cada característica a las dos primeras componentes principales')
    plt.grid(True)
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
    plt.xlim(-1, 1)
    plt.ylim(-1, 1)
plt.show()

### 8. Creamos un DataFrame con las puntuaciones de PCA para análisis adicionales

In [ ]:
pca_df = pd.DataFrame(
    X_pca_optimal,
    columns=[f'PC{i+1}' for i in range(X_pca_optimal.shape[1])]
)
pca_df['PRICE'] = y

In [ ]:
pca_df.head()

### 9. Correlación entre las componentes principales y el precio

In [ ]:
correlations = pca_df.corr()['PRICE'].drop('PRICE')
correlations

Visualizamos las correlaciones:

In [ ]:
plt.figure(figsize=(10, 6))
correlations.plot(kind='bar')
plt.title('Correlación entre las Componentes Principales y el Precio')
plt.ylabel('Coeficiente de Correlación')
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

### 10. Reconstruimos los datos originales a partir de las componentes principales

In [ ]:
X_reconstructed = pca_optimal.inverse_transform(X_pca_optimal)
X_reconstructed = scaler.inverse_transform(X_reconstructed)

Calculamos el error (RMSE) de reconstrucción:

In [ ]:
reconstruction_error = np.mean((X - X_reconstructed) ** 2)
np.sqrt(reconstruction_error)

Para una muestra comparamos los valores originales y reconstruidos:

In [ ]:
sample_idx = 0
comparison_df = pd.DataFrame({
    'Original': X.iloc[sample_idx],
    'Reconstruido': X_reconstructed[sample_idx],
    'Diferencia': X.iloc[sample_idx].values - X_reconstructed[sample_idx]
})
comparison_df

### Conclusiones del análisis PCA

In [ ]:
print(f'- Hemos reducido la dimensionalidad de {X.shape[1]} a {n_components} componentes.')
print(f'- Estas {n_components} componentes explican el {cumulative_variance_ratio[n_components-1]:.2%} de la varianza total.')
print('- La PCA1 está más correlacionada con {}.'.format(feature_names[np.argmax(abs(loadings[0]))]))
print('- La segunda componente principal está más correlacionada con {}.'.format(feature_names[np.argmax(abs(loadings[1]))]))

In [ ]:
dominant_features = {f'PC{i+1}': loadings_df[f'PC{i+1}'].abs().idxmax() for i in range(n_components)}
print(dominant_features)